<a href="https://colab.research.google.com/github/RDGopal/IB9AU-2026/blob/main/MLM5_Flow_Matching_1D_Teaching_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 Flow Matching in 1D — A Deterministic Alternative to Diffusion

This notebook builds a **1D Flow Matching** (FM) model — a deterministic ODE-based generative model — on a **multimodal** data distribution.

We will:
1) Create a **quad‑modal** 1D data distribution and visualize it.  
2) Define a **flow matching** training objective with a **linear probability path**.  
3) Train a tiny MLP to **predict the velocity field**
$v_\theta(x,t)$.  
4) **Sample** by integrating the ODE
$\dot x = v_\theta(x,t)$ from base noise to data.  
5) Compare generated samples to the original data.

> **Why Flow Matching?** Unlike diffusion (which trains via **noise prediction** and samples via a **reverse SDE/ODE**), flow matching trains by **regressing a velocity field** along a pre-defined path between a **base** distribution $p_0$ and the **data** distribution $p_1$; sampling solves a **deterministic ODE** from $t=0$ to $t=1$.



## Setup
We rely on PyTorch for a tiny MLP and Matplotlib for plots.

In [ ]:
import math, random, numpy as np
import torch, torch.nn as nn, torch.nn.functional as F
from torch import optim
import matplotlib.pyplot as plt

torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

## Data: a **quad‑modal** 1D distribution

We will use the same mixture of Gaussians as in the diffusion notebook for apples‑to‑apples comparison.


In [ ]:
# Mixture of Gaussians (quad‑modal) for p1(x)
weights = np.array([0.20, 0.30, 0.10, 0.40])  # sums to 1
means   = np.array([-6.0, -2.0,  2.0,  6.0])
stds    = np.array([ 0.5,  0.3,  0.4,  0.6])
assert np.isclose(weights.sum(), 1.0)

N = 50000  # data points
comps = np.random.choice(len(weights), size=N, p=weights)
x_data = np.random.normal(loc=means[comps], scale=stds[comps]).astype(np.float32)

def plot_hist(data, title, bins=200, range_=(-10,10)):
    plt.figure()
    plt.hist(data, bins=bins, range=range_, density=True)
    plt.title(title)
    plt.xlabel("x")
    plt.ylabel("density")
    plt.show()

plot_hist(x_data, "Target data distribution p1(x) (quad‑modal)")

## Flow Matching (FM) with a **linear path**

We set a base distribution $p_0(x) = \mathcal{N}(0,1)$, a data distribution $p_1$ (the mixture above), and define a **linear interpolant**:

$x_t = (1-t)\,x_0 + t\,x_1, \quad t\in[0,1]$,
where $x_0\sim p_0$ and $x_1\sim p_1$.

The **conditional** velocity field along this path is simply:
$v^*(x_t,t\mid x_0,x_1) = \tfrac{d}{dt} x_t = x_1 - x_0$.

**Training objective (Conditional Flow Matching):**
sample $x_0,x_1,t$, compute $x_t$, and regress a model $v_\theta(x_t,t)$ to the target $(x_1 - x_0)$ with mean‑squared error.


In [ ]:
# Build on-the-fly training samples for Conditional Flow Matching (linear path)
device = torch.device('cpu')

def sample_pairs(n):
    # base p0 ~ N(0,1)
    x0 = torch.randn(n, 1, device=device)
    # data p1 ~ mixture (sampled above)
    x1_np = np.random.choice(x_data, size=n, replace=True)
    x1 = torch.from_numpy(x1_np).float().unsqueeze(1).to(device)
    # time t ~ Uniform[0,1]
    t = torch.rand(n, device=device)
    # linear path: x_t = (1-t)*x0 + t*x1
    xt = (1.0 - t).unsqueeze(1) * x0 + t.unsqueeze(1) * x1
    # target velocity: v* = x1 - x0
    v_star = x1 - x0
    return xt, t, v_star

In [ ]:
import pandas as pd

# Generate a batch of training data
xt, t, vstar = sample_pairs(1024)

# Convert tensors to numpy arrays and then to a pandas DataFrame
training_data_df = pd.DataFrame({
    'xt': xt.squeeze().numpy(),
    't': t.squeeze().numpy(),
    'vstar': vstar.squeeze().numpy()
})

# Display the first few rows of the DataFrame
display(training_data_df.head())

## A tiny network to predict the velocity $v_\theta(x,t)$

We use a small MLP with a sinusoidal **time embedding**, just like in the diffusion notebook.


In [ ]:
# Sinusoidal time embedding
class TimeEmbedding(nn.Module):
    def __init__(self, dim=32, max_period=10000):
        super().__init__()
        self.dim = dim
        self.max_period = max_period
    def forward(self, t):  # t: [B] in [0,1]
        device = t.device
        half = self.dim // 2
        freqs = torch.exp(-math.log(self.max_period) * torch.arange(half, device=device) / half)
        args = (t.float().unsqueeze(1)) * freqs.unsqueeze(0)
        emb = torch.cat([torch.sin(args), torch.cos(args)], dim=1)
        if self.dim % 2 == 1:
            emb = torch.cat([emb, torch.zeros_like(emb[:, :1])], dim=1)
        return emb

class VelocityNet(nn.Module):
    def __init__(self, t_dim=32, hidden=64):
        super().__init__()
        self.tembed = TimeEmbedding(dim=t_dim)
        self.net = nn.Sequential(
            nn.Linear(1 + t_dim, hidden),
            nn.SiLU(),
            nn.Linear(hidden, hidden),
            nn.SiLU(),
            nn.Linear(hidden, 1),
        )
    def forward(self, x_t, t):
        te = self.tembed(t)
        h = torch.cat([x_t, te], dim=1)
        return self.net(h)

model = VelocityNet().to(device)
opt = optim.AdamW(model.parameters(), lr=2e-3)

## Train with Conditional Flow Matching (CFM)

Minimize $\mathbb{E}\,\|v_\theta(x_t,t) - (x_1-x_0)\|^2$ with batches of tuples $(x_t, t, x_1 - x_0)$.


In [ ]:
losses = []
for step in range(2500):  # quick CPU-friendly run
    xt, t, vstar = sample_pairs(1024)
    pred = model(xt, t)
    loss = F.mse_loss(pred, vstar)
    opt.zero_grad(); loss.backward(); opt.step()
    if (step+1) % 250 == 0:
        losses.append(float(loss.detach().cpu()))
        print(f"step {step+1}: loss {loss.item():.6f}")

plt.figure()
plt.plot(losses)
plt.title("Training loss (every 250 steps)")
plt.xlabel("checkpoint")
plt.ylabel("MSE")
plt.show()

## Sampling by integrating the ODE

After training, we generate samples by solving the ODE


$\frac{dx}{dt} = v_\theta(x,t)$ from $t=0$ to $t=1$.


We will use Euler and Heun (improved Euler) integrators.


In [ ]:
@torch.no_grad()
def sample_flow(n_samples=20000, steps=200, method='heun'):
    x = torch.randn(n_samples, 1)  # x(0) ~ p0
    dt = 1.0 / steps
    for s in range(steps):
        t = torch.full((n_samples,), (s/steps))
        v = model(x, t)
        if method == 'euler':
            x = x + dt * v
        else:  # Heun (predictor-corrector)
            x_pred = x + dt * v
            t_next = torch.full((n_samples,), ((s+1)/steps))
            v_next = model(x_pred, t_next)
            x = x + 0.5 * dt * (v + v_next)
    return x.squeeze(1).numpy()

x_gen_euler = sample_flow(method='euler')
x_gen_heun  = sample_flow(method='heun')

plot_hist(x_gen_euler, "Generated (Euler ODE, T=200)")
plot_hist(x_gen_heun,  "Generated (Heun ODE,  T=200)")
plot_hist(x_data,      "Reference data p1(x) — for comparison")

## Visualize the learned velocity field

We can examine $v_\theta(x,t)$ over a grid of $x$ values for a few $t$ snapshots.


In [ ]:
@torch.no_grad()
def plot_velocity_slices(xs=np.linspace(-10,10,200).astype(np.float32), ts=[0.0, 0.25, 0.5, 0.75, 1.0]):
    xs_torch = torch.from_numpy(xs).unsqueeze(1)
    import matplotlib.pyplot as plt
    plt.figure()
    for tt in ts:
        t = torch.full((len(xs),), tt)
        v = model(xs_torch, t).squeeze(1).numpy()
        plt.plot(xs, v, label=f"t={tt:.2f}")
    plt.title("Learned velocity slices v_θ(x,t)")
    plt.xlabel("x"); plt.ylabel("velocity")
    plt.legend(); plt.show()

plot_velocity_slices()

## Recap — What changed vs. diffusion?

- **Objective:** Diffusion learns to **predict noise**; Flow Matching learns to **predict velocity** along a path.  
- **Dynamics:** Diffusion uses an SDE/ODE; FM uses a **deterministic ODE**.  
- **Training data:** We pair $x_0\sim p_0$, $x_1\sim p_1$, sample $t\sim\mathcal U[0,1]$, compute $x_t$ and supervise $(x_1 - x_0)$.  
- **Sampling:** Integrate $\dot x = v_\theta(x,t)$ from $t=0$ (noise) to $t=1$ (data).


# Optional Tasks

## 1. 2D Distribution Generation with Flow Matching

In [ ]:
import tqdm
import math
import torch
import numpy as np
from torch import nn
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

# Parameters
N = 1000  # Number of points to sample
x_min, x_max = -4, 4
y_min, y_max = -4, 4
resolution = 100  # Resolution of the grid

# Create the grid
x = np.linspace(x_min, x_max, resolution)
y = np.linspace(y_min, y_max, resolution)
X, Y = np.meshgrid(x, y)

# Checkerboard pattern
length = 4
checkerboard = np.indices((length, length)).sum(axis=0) % 2

# Sample points in regions where checkerboard pattern is 1
sampled_points = []
while len(sampled_points) < N:
    # Randomly sample a point within the x and y range
    x_sample = np.random.uniform(x_min, x_max)
    y_sample = np.random.uniform(y_min, y_max)

    # Determine the closest grid index
    i = int((x_sample - x_min) / (x_max - x_min) * length)
    j = int((y_sample - y_min) / (y_max - y_min) * length)

    # Check if the sampled point is in a region where checkerboard == 1
    if checkerboard[j, i] == 1:
        sampled_points.append((x_sample, y_sample))

# Convert to NumPy array for easier plotting
sampled_points = np.array(sampled_points)

# Plot the checkerboard pattern
plt.figure(figsize=(6, 6))
plt.imshow(checkerboard, extent=(x_min, x_max, y_min, y_max), origin="lower", cmap=ListedColormap(["purple", "yellow"]))

# Plot sampled points
plt.scatter(sampled_points[:, 0], sampled_points[:, 1], color="red", marker="o")
plt.xlabel("X-axis")
plt.ylabel("Y-axis")
plt.show()

## 2. Exploring Different Probability Paths in Flow Matching

**Objective:** To understand how the choice of the probability path impacts the training and performance of a 1D Flow Matching model, and to relate this concept to practical considerations in generative modeling.

**Background:** In this notebook, we used a simple linear probability path ($x_t = (1-t)x_0 + tx_1$) to train the Flow Matching model. However, other paths can be used, such as those derived from Optimal Transport. The choice of path influences the target velocity field that the model learns to predict.

**Task:**

1.  **Review the Linear Path:** Briefly review the concept of the linear path and how the target velocity $(x_1 - x_0)$ is derived. Discuss why this target is used in the Conditional Flow Matching objective.

2.  **Introduce an Alternative Path (e.g., Quadratic Path):** Consider a simple non-linear path, for instance, a quadratic path:
    $x_t = (1 - t)^2 x_0 + (1 - (1-t)^2) x_1$
    *   Calculate the conditional velocity field $v^*(x_t, t \mid x_0, x_1)$ for this quadratic path by taking the derivative with respect to `t`.
    *   Discuss how this velocity field differs from the linear path's constant velocity.

3.  **Modify the Code:** Adapt the `sample_pairs` function in the provided notebook to generate training data for the quadratic path. You will need to:
    *   Implement the quadratic path formula to calculate `xt`.
    *   Implement the newly derived conditional velocity formula to calculate the target `v_star`.

4.  **Train a New Model:**
    *   Instantiate a *new* `VelocityNet` model. It's important to train a separate model for this new path.
    *   Train this new model using the modified `sample_pairs` function and the same training loop as before (you might need to adjust the number of training steps or learning rate).

5.  **Sample and Evaluate:**
    *   Use the trained model for the quadratic path to generate samples using the `sample_flow` function.
    *   Compare the generated samples from the quadratic path model to the original data distribution and the samples generated by the linear path model (using histograms).

6.  **Discuss the Findings (Practical Relevance):**
    *   Compare the training convergence and the quality of the generated samples for both the linear and quadratic paths. Which path seems to work better for this specific 1D multimodal data?
    *   Discuss the trade-offs of using different paths. Consider computational cost (is the target velocity easy to compute?), smoothness of the velocity field (does a smoother field make learning easier?), and theoretical guarantees (if any).
    *   **Practical Connection:** How might the choice of probability path be relevant in real-world applications of generative modeling (e.g., image generation, text-to-speech)? Consider scenarios where a specific type of transformation or interpolation between noise and data might be desired or more efficient. For example, in image generation, would you want a linear interpolation in pixel space, or something that follows a more perceptually relevant path?
